### Ingest math and language arts proficiency scores

In [2]:
# Look at columns in files one at a time before we upload to worm
import os,sys,pandas as pd, numpy as np,seaborn as sns

In [12]:
os.chdir("/home/bb/datasets/")
for root, dirs, files in os.walk('./state_achievement_math_reading/',topdown=False):
    for f in files:
        print(f)

math-achievement-sch-sy2017-18.csv
math-achievement-sch-sy2012-13.csv
rla-achievement-sch-sy2018-19-wide.csv
rla-achievement-sch-sy2013-14.csv
math-achievement-sch-sy2014-15.csv
rla-achievement-sch-sy2016-17.csv
math-achievement-sch-sy2009-10.csv
math-achievement-sch-sy2015-16.csv
rla-achievement-sch-sy2015-16.csv
math-achievement-sch-sy2018-19-wide.csv
rla-achievement-sch-sy-2010-11.csv
math-achievement-sch-sy2011-12.csv
math-achievement-sch-sy2016-17.csv
rla-achievement-sch-sy-2011-12.csv
rla-achievement-sch-sy2009-10.csv
math-achievement-sch-sy2010-11.csv
math-achievement-sch-sy2013-14.csv
rla-achievement-sch-sy2017-18.csv
rla-achievement-sch-sy2014-15.csv
rla-achievement-sch-sy2012-13.csv
